In [ ]:
from ipywidgets import GridspecLayout, Output
from IPython.display import display

import numpy as np
import matplotlib.pyplot as plt
import sys
import scipy
import scipy.misc
import time
import scipy.ndimage
import random
import torch
import torchvision
from torchvision.transforms.functional import normalize
import matplotlib.cm as cm
sys.path.insert(0, '../src')

In [ ]:
from ebconv.splines import BSplineElement
from ebconv.kernel import CardinalBSplineKernel, create_random_centers
from ebconv.utils import round_modf, tensordot
from ebconv.nn.functional import cbsconv

# BSplines examples

In [ ]:
def b_splines_example():
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(21, 5))
    x = np.linspace(-8, 8, 1000)
    s = np.zeros_like(x)
    for c in np.arange(-4, 4 + 1):
        b = BSplineElement.create_cardinal(c)
        y = b(x)
        s += y
        ax1.plot(x, y)
    ax1.plot(x, s, label='BSplines sum', linewidth=1.5)
    ax1.grid()
    ax1.legend()
    ax1.set_title("Standard basis on interval")

    x = np.linspace(-5, 5, 1000)
    for s, c, n in zip((1, 2, 0.5), (-0.3, -1.7, 2.5), (2, 3, 4)):
        b = BSplineElement.create_cardinal(c, s, n)
        ax2.plot(x, b(x), label=f'c={c}, s={s}, n={n}')
    ax2.grid()
    ax2.legend()
    ax2.set_title("Different parameters")
    plt.show()

b_splines_example()

# 2D BSpline Kernel basis

In [ ]:
def bsplinekernel_example(N=7, s=10, k=3):
    """Example of how to use the BSplineKernelBasis.
    
    This is a simple example on how to generate 
    a set of randomly centered basis spline functions.
    """
    random.seed(42)
    # 2d interval, sample from top left to bottom right
    # in this way the value (0, 0) corresponds to (-100, 100).
    interval = np.array((100, -100, -100, 100)).reshape(2, 2)
    c = create_random_centers(interval, N)
    
    kb = CardinalBSplineKernel.create(c=c, s=s, k=k)
    bounds = interval
    bounds = kb.centered_bounds()
    axb = bounds.flatten()

    # We need to flip the axes boundaries because
    # we want to sample from top left to bottom left
    # and from top left to top right
    bounds = bounds.T
    
    # Now we generate three sets of random weights
    w = np.random.rand(3, N) * 2 - 1
    
    # Generate the meshgrid
    np.meshgrid(*np.linspace(*bounds, 200).T)
    xx, yy = np.meshgrid(*np.linspace(*bounds, 200).T)
    
    # Compute the linear combination of bases
    tb = kb(xx, yy)
    k = np.tensordot(w, tb, axes=1).T
    k = np.interp(k, (k.min(), k.max()), (0, 1))
    
    fig, axes = plt.subplots(1, 4, figsize=(25, 8))
    axes[0].scatter(*kb.c.T, marker='x', color='white')
    axes[0].imshow(k, extent=axb, origin='lower')
    axes[0].set_title('All channels')
    k_channels = (k[..., 0], k[..., 1], k[..., 2])
    for ax, c, cmap in zip(axes[1:], k_channels, ('Reds', 'Greens', 'Blues')):
        ax.scatter(*kb.c.T, marker='x', color='white')
        ax.imshow(c, extent=axb, origin='lower', cmap=cmap, vmin=0, vmax=1)
        ax.set_title(cmap)
    plt.show()
    return kb, w, bounds

kb, w, interval = bsplinekernel_example()

# Example of a 2d convolution

In [ ]:
def standard_convolution(img, kb, w, interval):
    # We sample a 2d kernel
    linspaces = []
    bounds = kb.centered_bounds()
    x_interval, y_interval = bounds
    kernel_size = bounds
    kernel_size = np.ceil(kernel_size[:, 1] - kernel_size[:, 0]).astype(int)
    
    x = np.linspace(*x_interval, kernel_size[0])
    y = np.linspace(*y_interval, kernel_size[1])
    xx, yy = np.meshgrid(x, y)

    # Construct the per channel kernel weights
    k = torch.Tensor(np.tensordot(w, kb(xx, yy), axes=1).T)
    weights = torch.stack((k[:, :, 0][None, :], 
                           k[:, :, 1][None, :], 
                           k[:, :, 2][None, :]))
    conv = torch.nn.functional.conv2d(img, weights, groups=3)
    
    conv = conv.numpy().squeeze()
    conv = np.moveaxis(conv, 0, -1)
    conv = np.interp(conv, (conv.min(), conv.max()), (0, 1))
    return conv

def bspline_convolution(img, kb, w, interval):
    oC, n = w.shape
    w = torch.Tensor(w).reshape(oC, 1, n)
    conv = cbsconv(img, None, w, kb.c, kb.s, kb.k, groups=3)
    conv = conv.numpy().squeeze()
    conv = np.moveaxis(conv, 0, -1)
    conv = np.interp(conv, (conv.min(), conv.max()), (0, 1))
    return conv
    
def convolutions_example(kb, w, interval):
    # Our test image
    img = scipy.misc.face()
    img = torch.Tensor(np.moveaxis(img, -1, 0))
    #img = normalize(img, (0, 0, 0), (100, 100, 100))
    img = img[None, :]
    
    start_time = time.time()
    conv_standard = standard_convolution(img, kb, w, interval)
    t_standard = time.time() - start_time
    
    start_time = time.time()
    conv_bspline = bspline_convolution(img, kb, w, interval)
    t_bspline = time.time() - start_time
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(25, 25))
    ax1.set_title('Standard convolution (%.2fs)' % t_standard)
    ax1.imshow(conv_standard)
    ax2.set_title('Spline convolution (%.2fs)' % t_bspline)
    ax2.imshow(conv_bspline)
    plt.show()
    
_ = convolutions_example(kb, w, interval)
